In [36]:
import pandas as pd
import numpy as np

In [37]:
# Lista de los valores únicos

df=pd.read_csv("../data/interim/modelado_FINAL.csv")

df.columns

Index(['id', 'title', 'description', 'location_area', 'company_display_name',
       'category_tag', 'contract_time', 'salary_min', 'salary_max',
       'salary_is_predicted', 'created', 'latitude', 'longitude',
       'redirect_url', 'is_good', 'description_full', 'remote',
       'permanent_contract', 'freelance', 'bonuses', 'career_development',
       'immediate_start', 'startup', 'multinational',
       'consulting_or_outsourcing', 'experience_gt_5', 'intern', 'junior',
       'senior', 'lead', 'principal_or_manager', 'higher_education',
       'certifications_required', 'software_development', 'data_science_ml',
       'cybersecurity', 'qa_testing', 'it_support_infrastructure',
       'project_product_management', 'ux_ui_design', 'intern_title',
       'junior_title', 'senior_title', 'lead_title',
       'principal_or_manager_title'],
      dtype='object')

In [38]:
columns_to_drop = ['description', 'category_tag', 'salary_is_predicted', 'is_good']
df = df.drop(columns=columns_to_drop)

In [39]:
df.columns.to_list()

['id',
 'title',
 'location_area',
 'company_display_name',
 'contract_time',
 'salary_min',
 'salary_max',
 'created',
 'latitude',
 'longitude',
 'redirect_url',
 'description_full',
 'remote',
 'permanent_contract',
 'freelance',
 'bonuses',
 'career_development',
 'immediate_start',
 'startup',
 'multinational',
 'consulting_or_outsourcing',
 'experience_gt_5',
 'intern',
 'junior',
 'senior',
 'lead',
 'principal_or_manager',
 'higher_education',
 'certifications_required',
 'software_development',
 'data_science_ml',
 'cybersecurity',
 'qa_testing',
 'it_support_infrastructure',
 'project_product_management',
 'ux_ui_design',
 'intern_title',
 'junior_title',
 'senior_title',
 'lead_title',
 'principal_or_manager_title']

## Limpiando el seniority

In [40]:
# Crear la columna salary_avg como media de salary_min y salary_max
df["salary_avg"] = df[["salary_min", "salary_max"]].mean(axis=1)

# Columnas relacionadas con seniority (scores y títulos)
seniority_levels = ["intern", "junior", "senior", "lead", "principal_or_manager"]
title_columns = [f"{level}_title" for level in seniority_levels]

# Calcular la media de salary_avg por cada nivel de seniority, cuando ese nivel es el dominante
salary_avg_means = {}
for level in seniority_levels:
    # Selecciona filas donde ese nivel es el dominante (valor más alto)
    is_dominant = df[level] >= df[[lvl for lvl in seniority_levels if lvl != level]].max(axis=1)
    salary_avg_means[level] = df[is_dominant]["salary_avg"].mean()

# Mapeo de jerarquía
seniority_ranks = {level: i for i, level in enumerate(seniority_levels)}

# Función para determinar el seniority
def determine_seniority(row):
    # Si algún título es 1, elegimos el de mayor jerarquía
    titles = {level: row[f"{level}_title"] for level in seniority_levels}
    active_titles = [level for level, val in titles.items() if val == 1]
    if active_titles:
        return max(active_titles, key=lambda x: seniority_ranks[x])
    
    # Si no hay título, usamos los scores
    scores = {level: row[level] for level in seniority_levels}
    max_level = max(scores, key=scores.get)
    max_score = scores[max_level]
    second_max = max([v for k, v in scores.items() if k != max_level], default=0)

    # Si hay una diferencia clara en los scores
    if max_score - second_max > 0.2:
        return max_level
    else:
        # Comparar salary_avg con las medias por nivel
        if pd.isna(row["salary_avg"]):
            return max_level  # no se puede comparar si salary_avg es NaN

        # Solo niveles con media disponible
        diffs = {
            level: abs(row["salary_avg"] - salary_avg_means[level])
            for level in seniority_levels
            if not pd.isna(salary_avg_means[level])
        }

        if not diffs:
            return max_level  # fallback si no hay medias disponibles

        min_diff = min(diffs.values())
        closest_levels = [level for level, diff in diffs.items() if diff == min_diff]

        # Elegir el de mayor jerarquía si hay empate
        return max(closest_levels, key=lambda x: seniority_ranks[x])


# Aplicar la función
df["seniority"] = df.apply(determine_seniority, axis=1)


In [41]:
columns_to_drop = ["intern", "junior", "senior", "lead", "principal_or_manager", "intern_title", "junior_title", "senior_title", "lead_title", "principal_or_manager_title"]
df = df.drop(columns=columns_to_drop)

In [42]:
df.columns

Index(['id', 'title', 'location_area', 'company_display_name', 'contract_time',
       'salary_min', 'salary_max', 'created', 'latitude', 'longitude',
       'redirect_url', 'description_full', 'remote', 'permanent_contract',
       'freelance', 'bonuses', 'career_development', 'immediate_start',
       'startup', 'multinational', 'consulting_or_outsourcing',
       'experience_gt_5', 'higher_education', 'certifications_required',
       'software_development', 'data_science_ml', 'cybersecurity',
       'qa_testing', 'it_support_infrastructure', 'project_product_management',
       'ux_ui_design', 'salary_avg', 'seniority'],
      dtype='object')

## Dividiendo location_area

In [43]:
df["location_area"]

0          US, Ohio, Franklin County, Grandview Heights
1      US, Massachusetts, Middlesex County, Marlborough
2                  US, New York, Orange County, Chester
3                 US, Wisconsin, Kenosha County, Trevor
4              US, Wisconsin, Racine County, Burlington
5                US, Illinois, Lake County, Great Lakes
6         US, Illinois, Lake County, Lincolnshire Woods
7                US, Wisconsin, Kenosha County, Kenosha
8                       US, Illinois, Lake County, Park
9            US, Illinois, Lake County, Winthrop Harbor
10          US, California, San Diego County, Escondido
11                US, Washington, King County, Westwood
12                                                   US
13                US, Virginia, Fairfax County, Herndon
14          US, California, San Diego County, Oceanside
15                                                   US
16                        US, California, San Francisco
17                                              

In [ ]:
# Eliminar 'US, ' de la columna location_area
df['location_area'] = df['location_area'].str.replace('US, ', '', regex=False)

# Dividir la columna location_area en partes, separadas por comas
split_location = df['location_area'].str.split(',', expand=True)

# Asegurarse de que las filas con menos de tres partes tengan valores NaN o None
split_location = split_location.rename(columns={0: 'state', 1: 'county', 2: 'city'})

# Limpiar los espacios en blanco en cada nueva columna
split_location['state'] = split_location['state'].str.strip()
split_location['county'] = split_location['county'].str.strip()
split_location['city'] = split_location['city'].str.strip()

# Reemplazar valores vacíos con None
split_location['state'] = split_location['state'].replace('', None)
split_location['county'] = split_location['county'].replace('', None)
split_location['city'] = split_location['city'].replace('', None)

# Unir las columnas resultantes de nuevo al dataframe original
df = df.join(split_location)



In [50]:
df = df.drop('location_area', axis=1)

In [51]:
df.columns

Index(['id', 'title', 'company_display_name', 'contract_time', 'salary_min',
       'salary_max', 'created', 'latitude', 'longitude', 'redirect_url',
       'description_full', 'remote', 'permanent_contract', 'freelance',
       'bonuses', 'career_development', 'immediate_start', 'startup',
       'multinational', 'consulting_or_outsourcing', 'experience_gt_5',
       'higher_education', 'certifications_required', 'software_development',
       'data_science_ml', 'cybersecurity', 'qa_testing',
       'it_support_infrastructure', 'project_product_management',
       'ux_ui_design', 'salary_avg', 'seniority', 'state', 'county', 'city'],
      dtype='object')

## Aplicando thresholds

In [ ]:
bonus_threshold = 0.9
career_development_threshold = 0.9
freelance_threshold = 0.6
